## The purpose of this notebook is to cycle through all acquired CSV files for BTS On-Time Performance Data, cast columns to appropriate data types, and remove cancelled and diverted flights (since we are only concerned with delays). It then writes the files to a sqlite database and to a parquet file.

In [1]:
# Import libraries
import pandas as pd
import sqlite3
from glob import glob
from datetime import datetime
pd.set_option('display.max_columns', None)

In [2]:
usecols = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'Tail_Number',
'Flight_Number_Reporting_Airline', 'OriginCityMarketID', 'Origin', 'DestCityMarketID', 'Dest', 'CRSDepTime',
'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 
'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk',
'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance',
'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'DivAirportLandings']

# File name for testing
# fname = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_1.csv'

# Function to read individual CSV into dataframe and clean for further use
def getCleanDF(fname, usecols=usecols, removeCanDiv=False):
    # Read CSV into PDF
    df = pd.read_csv(fname, usecols=usecols, keep_default_na=False, 
                     parse_dates = ['FlightDate'], low_memory=False, dtype=str)

    # Coerce certain columns to integers
    for col in ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
                'Flight_Number_Reporting_Airline','OriginCityMarketID']:
        df[col] = df[col].astype(int)

    # Fix null values and coerce additional columns to integers
    for col in ['DepDelay', 'DepDelayMinutes', 'DepDel15', 'TaxiOut', 'TaxiIn', 'ArrDelay', 
                    'ArrDelayMinutes','ArrDel15', 'Cancelled', 'Diverted', 'CRSElapsedTime', 
                    'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'CarrierDelay', 
                    'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'DivAirportLandings']:
        df[col] = df[col].replace('',0)
        df[col] = df[col].astype(float)
        df[col] = df[col].apply(lambda x: int(x))

    # Function to convert time string into time object
    def formatTime(t):
        if t=='':
            return None
        elif t=='2400':
            t='0000'
        return datetime.strptime(t, '%H%M').time()

    # Coerce certain columns to time
    for col in ['CRSDepTime', 'DepTime', 'WheelsOff', 'WheelsOn', 'CRSArrTime', 'ArrTime']:
        df[col] = df[col].apply(lambda t: formatTime(t))
    
    # print('Records processed:', len(df), f"({len(df.columns)} columns)")
    
    # Filter out cancellations and diversions if desired
    if removeCanDiv==True and 'Cancelled' in df.columns and 'Diverted' in df.columns:
        df = df[df.Cancelled==0]
        df = df[df.Diverted==0]
        df = df.drop(['Cancelled', 'Diverted', 'CancellationCode', 'DivAirportLandings'], axis=1)
        # print('Records remaining after filtering', len(df), f"({len(df.columns)} columns)")
        
    return df

#df = getCleanDF(fname, usecols, True)

In [6]:
# Connect to sqlite database file
conn = sqlite3.connect('bts_on_time_v3.db')

num_records = 0
num_files = 0
start = datetime.now()

# Could do this using a glob, but this keeps the records in date order
for yr in range(2015,2024):
    for mo in range(1,13):
        try:
            base = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_'
            fname = base + str(yr) + '_' + str(mo) + '.csv'
            cdf = getCleanDF(fname)
            cdf.to_sql('on_time', conn, if_exists='append', index=False)
            print(fname, f"({len(cdf)} x {len(cdf.columns)})")
            num_records += len(cdf)
            num_files += 1
        except:
            end = datetime.now()
            print(f'Finished processing! Total: {num_records} records from {num_files} files')
            print(f'Time elapsed: {end-start}')
            break
            

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_1.csv (469968 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_2.csv (429191 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_3.csv (504312 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_4.csv (485151 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_5.csv (496993 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_6.csv (503897 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_7.csv (520718 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_8.csv (510536 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_9.csv (464946 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_10.csv (486165 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2015_11.csv (467972 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_12.csv (557095 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_1.csv (538837 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_2.csv (502749 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_3.csv (580322 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_4.csv (561441 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_5.csv (579958 x 46)
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2023_6.csv (577262 x 46)
Finished processing! Total: 52493035 from 102
Time elapsed: 0:31:23.809810


In [9]:
#Write contents of sqlite table to parquet

#!pip install pyarrow
import pyarrow as pa
import pyarrow.parquet as pq

conn = sqlite3.connect('bts_on_time_v3.db')
query = 'select * from on_time'
chunk = 2000000
start = datetime.now()

for i, df in enumerate(pd.read_sql(query, con=conn, chunksize=chunk)):
    table = pa.Table.from_pandas(df)
    if i == 0:
        pqwriter = pq.ParquetWriter('bts_on_time_v3.parquet', table.schema)
    pqwriter.write_table(table)
    print('Chunk Written:', i+1)
if pqwriter:
    pqwriter.close()

end = datetime.now()
print(f"Finished processing! Time elapsed: {end-start}")

Chunk Written: 1
Chunk Written: 2
Chunk Written: 3
Chunk Written: 4
Chunk Written: 5
Chunk Written: 6
Chunk Written: 7
Chunk Written: 8
Chunk Written: 9
Chunk Written: 10
Chunk Written: 11
Chunk Written: 12
Chunk Written: 13
Chunk Written: 14
Chunk Written: 15
Chunk Written: 16
Chunk Written: 17
Chunk Written: 18
Chunk Written: 19
Chunk Written: 20
Chunk Written: 21
Chunk Written: 22
Chunk Written: 23
Chunk Written: 24
Chunk Written: 25
Chunk Written: 26
Chunk Written: 27
Finished processing! Time elapsed: -1 day, 23:47:44.873911


In [10]:
# Identify unique tail numbers from the BTS data
df_tail = pd.read_sql('select distinct Tail_Number from on_time', conn)

# Create a structure to allow for the presence of the N in the Tail_Number or not, to be
# compatible with the FAA data
df_tail['Tail2'] = df_tail.Tail_Number.apply(lambda x: x[1:])

df_compare = df_tail[['Tail_Number']]
df_compare2 = df_tail[['Tail2']]
df_compare2.columns=['Tail_Number']

df_compare = pd.concat([df_compare, df_compare2], axis=0, ignore_index=True)
df_compare

,Tail_Number
0,N787AA
1,N795AA
2,N788AA
3,N791AA
4,N783AA
...,...
17943,8879Q
17944,8882Q
17945,8869L
17946,762YX


In [14]:
# Read FAA data from sqlite table
conn2 = sqlite3.connect('faa_joined.db')
df_faa = pd.read_sql('select * from faa_joined', conn2)

# Only keep rows from FAA data that match up to a tail number in the BTS data
df_faa = df_faa.merge(df_compare, how='inner', left_on='N-NUMBER', right_on='Tail_Number')
df_faa = df_faa.drop('Tail_Number', axis=1)

# Drop columns that will not be useful
col_to_drop = ['STREET', 'STREET2', 'CITY', 'STATE', 'ZIP CODE', 
               'OTHER NAMES(1)', 'OTHER NAMES(2)', 'OTHER NAMES(3)', 'OTHER NAMES(4)',
               'OTHER NAMES(5)','KIT MFR', ' KIT MODEL']

df_faa = df_faa.drop(col_to_drop, axis=1)

df_faa.to_csv('faa_abridged.csv')

## Items below are to build some summary pivot tables

In [19]:
# Query for all delayed flights by tail number
query_delay = '''
select Tail_Number, count(*) as DelayedFlights from on_time
where DepDel15==1 or ArrDel15==1
group by Tail_Number order by Tail_Number asc
'''

# Query for all flights by tail number
query_flights = '''
select Tail_Number, count(*) as TotalFlights from on_time
group by Tail_Number order by Tail_Number asc
'''

df_delay = pd.read_sql(query_delay, conn)
df_flights = pd.read_sql(query_flights, conn)

# Combine
df_delay = df_delay.merge(df_flights, how='inner', on='Tail_Number')

df_delay

,Tail_Number,DelayedFlights,TotalFlights
0,190NV,200,788
1,191NV,195,716
2,192NV,148,480
3,193NV,247,742
4,194NV,179,604
...,...,...,...
8919,SS1,27,38
8920,SS2,7,23
8921,SS25,2,4
8922,SS3,9,10


In [22]:
# Create column to join with FAA data
df_delay['N-NUMBER'] = df_delay.Tail_Number.apply(lambda x: x[1:] if x[0].upper()=='N' else x)
df_faa_flights = df_delay.merge(df_faa, how='inner', on='N-NUMBER')
df_faa_flights['DelayedFrac'] = df_faa_flights.DelayedFlights/df_faa_flights.TotalFlights
df_faa_flights.to_csv('faa_abridged_w_flight_counts.csv')

In [30]:
# Pivot to identify delays by aircraft manufacturer
df_pivot = df_faa_flights.pivot_table(index='MFR_aircraft', values=['DelayedFlights', 'TotalFlights'], aggfunc='sum').reset_index()

df_pivot['DelayedFrac'] = df_pivot.DelayedFlights/df_pivot.TotalFlights
df_pivot.sort_values(by=['DelayedFrac'], axis=0, ascending=False, ignore_index=True, inplace=True)

df_pivot

,MFR_aircraft,DelayedFlights,TotalFlights,DelayedFrac
0,HUGHES,6,6,1.000000
1,WARREN RANDAL,1,1,1.000000
2,BELLANCA,17,42,0.404762
3,AIRBUS S A S,16,43,0.372093
4,PAIR MIKE E,328,955,0.343455
...,...,...,...,...
109,MCDONNELL DOUGLAS CORPORATION,20382,131440,0.155067
110,RAYTHEON CORPORATE JETS INC,246,1605,0.153271
111,ALPHA M SCIENTIFIC,254,1708,0.148712
112,SMITH ROLAND W,244,1662,0.146811


In [34]:
# Pivot to identify delays by engine manufacturer
df_pivot_e = df_faa_flights.pivot_table(index='MFR', values=['DelayedFlights', 'TotalFlights'], aggfunc='sum').reset_index()

df_pivot_e['DelayedFrac'] = df_pivot_e.DelayedFlights/df_pivot_e.TotalFlights
df_pivot_e.sort_values(by=['DelayedFrac'], axis=0, ascending=False, ignore_index=True, inplace=True)

df_pivot_e

,MFR,DelayedFlights,TotalFlights,DelayedFrac
0,ROLLS DEU,96,208,0.461538
1,TURBOMECA,236,670,0.352239
2,ROLLS-ROY,609,1746,0.348797
3,MATTITUCK,205,611,0.335516
4,U/A CANADA,339,1061,0.319510
5,WRIGHT,199,631,0.315372
6,THIELERT,746,2546,0.293009
7,P&W,50431,188988,0.266848
8,SUPERIOR,150,605,0.247934
9,IAE,1578034,6399265,0.246596
